In [31]:
import numpy as np

### Load Dataset

In [32]:
all_data = np.load('train_dev_test.npz')

In [33]:
train_encoder_output = all_data['train_encoder_output']
train_decoder_input = all_data['train_decoder_input']
train_decoder_target = all_data['train_decoder_target'][:,1:,:]
validation_encoder_output = all_data['validation_encoder_output']
validation_decoder_input = all_data['validation_decoder_input']
validation_decoder_target = all_data['validation_decoder_target'][:,1:,:]
test_encoder_output = all_data['test_encoder_output']
test_decoder_input = all_data['test_decoder_input']
test_decoder_target = all_data['test_decoder_target'][:,1:,:]

In [34]:
print("Train Encoder Output", train_encoder_output.shape)
print("Train Decoder Input", train_decoder_input.shape)
print("Train Decoder Target", train_decoder_target.shape)

Train Encoder Output (30000, 512)
Train Decoder Input (30000, 38)
Train Decoder Target (30000, 38, 2531)


In [35]:
from caption_utils import *
train_fns_list, dev_fns_list, test_fns_list = load_split_lists()

train_captions_raw, dev_captions_raw, test_captions_raw = get_caption_split()
vocab = create_vocab(train_captions_raw)
token2idx, idx2token = vocab_to_index(vocab)    
captions_data = (train_captions_raw.copy(), dev_captions_raw.copy(), test_captions_raw.copy())
train_captions, dev_captions, test_captions = process_captions(captions_data, token2idx)

## Model Architecture

In [36]:
from keras.models import Model, Sequential
from keras.layers import Input, Dense, BatchNormalization, RepeatVector, Concatenate, Merge, Masking
from keras.layers import LSTM, GRU, Embedding, TimeDistributed, Bidirectional
from keras import backend as K
from keras import optimizers
from keras.utils import plot_model

#### Parameters

In [37]:
emb_size = 300
lstm_size = 300
vocab_size = len(vocab)
max_length = train_decoder_target.shape[1]
lstm_layers = 2

lr = 0.001
dropout_rate = 0.2
batch_size = 32
n_epochs = 30

### Model

In [38]:
K.clear_session()

In [39]:
# Image -> Image embedding
image_input = Input(shape=(train_encoder_output.shape[1], ), name='image_input')
print("Image Input shape", image_input.shape)
img_emb = Dense(emb_size, activation='relu', name='img_embedding')(image_input)
print(img_emb.shape)

Image Input shape (?, 512)
(?, 300)


In [40]:
# Sentence to Word embedding
caption_inputs = Input(shape=(None, ), name='caption_input')
print("Caption Input Shape", caption_inputs.shape)
emb_layer = Embedding(input_dim=vocab_size, output_dim=emb_size, name='Embedding')
word_emb = emb_layer(caption_inputs)
print(word_emb.shape)

Caption Input Shape (?, ?)
(?, ?, 300)


In [41]:
decoder_cell = LSTM(lstm_size, return_sequences=True, return_state=True, name='decoder', dropout=dropout_rate, recurrent_dropout=dropout_rate)
encoder_states = [img_emb, img_emb]
decoder_out, _, _ = decoder_cell(word_emb, initial_state = encoder_states)

In [42]:
output_layer = TimeDistributed(Dense(vocab_size, activation='softmax'))
decoder_out = output_layer(decoder_out)
print(decoder_out.shape)

(?, ?, 2531)


In [43]:
rmsprop = optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=None, decay=1e-6)

In [44]:
model = Model(inputs=[image_input,caption_inputs], outputs=[decoder_out])

model.compile(optimizer=rmsprop,
               loss='categorical_crossentropy',
               metrics=['accuracy'])

In [45]:
model.summary()
#plot_model(model, to_file='model2.png', show_shapes=True)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
caption_input (InputLayer)      (None, None)         0                                            
__________________________________________________________________________________________________
image_input (InputLayer)        (None, 512)          0                                            
__________________________________________________________________________________________________
Embedding (Embedding)           (None, None, 300)    759300      caption_input[0][0]              
__________________________________________________________________________________________________
img_embedding (Dense)           (None, 300)          153900      image_input[0][0]                
__________________________________________________________________________________________________
decoder (L

In [46]:
from keras.callbacks import ModelCheckpoint

checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.VGG16.final.hdf5', 
                               verbose=1, save_best_only=True)

In [47]:
model.fit([train_encoder_output, train_decoder_input], [train_decoder_target], 
           validation_data=([validation_encoder_output, validation_decoder_input], [validation_decoder_target]),
           epochs=n_epochs, batch_size=batch_size, callbacks=[checkpointer], verbose=2)

Train on 30000 samples, validate on 5000 samples
Epoch 1/30
 - 82s - loss: 1.1760 - acc: 0.0974 - val_loss: 1.0501 - val_acc: 0.1105

Epoch 00001: val_loss improved from inf to 1.05011, saving model to saved_models/weights.best.VGG16.final.hdf5
Epoch 2/30


/home/ubuntu/anaconda3/envs/tensorflow-3.5/lib/python3.5/site-packages/keras/engine/topology.py:2368: UserWarning: Layer decoder was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'img_embedding/Relu:0' shape=(?, 300) dtype=float32>, <tf.Tensor 'img_embedding/Relu:0' shape=(?, 300) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  str(node.arguments) + '. They will not be included '


 - 81s - loss: 0.9920 - acc: 0.1163 - val_loss: 0.9902 - val_acc: 0.1178

Epoch 00002: val_loss improved from 1.05011 to 0.99022, saving model to saved_models/weights.best.VGG16.final.hdf5
Epoch 3/30
 - 81s - loss: 0.9372 - acc: 0.1231 - val_loss: 0.9664 - val_acc: 0.1206

Epoch 00003: val_loss improved from 0.99022 to 0.96639, saving model to saved_models/weights.best.VGG16.final.hdf5
Epoch 4/30
 - 81s - loss: 0.9054 - acc: 0.1273 - val_loss: 0.9547 - val_acc: 0.1222

Epoch 00004: val_loss improved from 0.96639 to 0.95472, saving model to saved_models/weights.best.VGG16.final.hdf5
Epoch 5/30
 - 81s - loss: 0.8835 - acc: 0.1307 - val_loss: 0.9492 - val_acc: 0.1220

Epoch 00005: val_loss improved from 0.95472 to 0.94920, saving model to saved_models/weights.best.VGG16.final.hdf5
Epoch 6/30
 - 81s - loss: 0.8652 - acc: 0.1332 - val_loss: 0.9426 - val_acc: 0.1235

Epoch 00006: val_loss improved from 0.94920 to 0.94260, saving model to saved_models/weights.best.VGG16.final.hdf5
Epoch 7/30


KeyboardInterrupt: 

## Inference

In [49]:
encoder_model = Model(image_input, encoder_states)

/home/ubuntu/anaconda3/envs/tensorflow-3.5/lib/python3.5/site-packages/keras/engine/topology.py:1541: UserWarning: The list of outputs passed to the model is redundant. All outputs should only appear once. Found: [<tf.Tensor 'img_embedding/Relu:0' shape=(?, 300) dtype=float32>, <tf.Tensor 'img_embedding/Relu:0' shape=(?, 300) dtype=float32>]
  ' Found: ' + str(self.outputs))


In [50]:
encoder_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
image_input (InputLayer)     (None, 512)               0         
_________________________________________________________________
img_embedding (Dense)        (None, 300)               153900    
Total params: 153,900
Trainable params: 153,900
Non-trainable params: 0
_________________________________________________________________


In [51]:
decoder_state_input_h = Input(shape=(lstm_size, ))
decoder_state_input_c = Input(shape=(lstm_size, ))

In [52]:
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_cell(word_emb, initial_state=decoder_states_inputs)

In [53]:
decoder_states = [state_h, state_c]
decoder_outputs = output_layer(decoder_outputs)
decoder_model = Model([caption_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)

In [54]:
decoder_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
caption_input (InputLayer)      (None, None)         0                                            
__________________________________________________________________________________________________
Embedding (Embedding)           (None, None, 300)    759300      caption_input[0][0]              
__________________________________________________________________________________________________
input_1 (InputLayer)            (None, 300)          0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 300)          0                                            
__________________________________________________________________________________________________
decoder (L

In [55]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence

## Load the Model with the Best Validation Loss

In [56]:
model.load_weights('saved_models/weights.best.VGG16.final.hdf5')

In [57]:
def generate_seq(img_input):
    if img_input.shape != (1, 512):
        img_input = img_input.reshape(1, 512)
    
    assert(img_input.shape == (1, 512))
    stop_condition = False
    decoded_sentence = []
    target_seq = np.array([token2idx['<bos>']]).reshape(1, 1)
    states_value = encoder_model.predict(img_input)

    while not stop_condition:

        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        sampled_token_index = int(np.argmax(output_tokens[0, -1, :]))

        sampled_char = idx2token[sampled_token_index]

        decoded_sentence += [sampled_char]

        if (sampled_char == '<eos>' or len(decoded_sentence) > 30):
            stop_condition = True

        target_seq = np.array([sampled_token_index]).reshape(1, 1)

        states_value = [h, c]

    return ' '.join(decoded_sentence)

In [58]:
for i in range(20):
    print(generate_seq(test_encoder_output[i*5, :]))
    print('*'*30)
    for j in range(5):
        print(intseq_to_caption(idx2token, test_captions[test_fns_list[i]][j]))
    print()

    

two dogs are running in the snow <eos>
******************************
<bos> the dogs are in the snow in front of a fence <eos>
<bos> the dogs play on the snow <eos>
<bos> two brown dogs playfully fight in the snow <eos>
<bos> two brown dogs wrestle in the snow <eos>
<bos> two dogs playing in the snow <eos>

a dog with a blue ball in its mouth is laying on the ground <eos>
******************************
<bos> a brown and white dog swimming towards some in the pool <eos>
<bos> a dog in a swimming pool swims toward <unk> <unk> <unk> see <eos>
<bos> a dog swims in a pool near a person <eos>
<bos> small dog is paddling through the water in a pool <eos>
<bos> the small brown and white dog is in the pool <eos>

a man in a green shirt and a woman in a white shirt and sunglasses is sitting on a bench <eos>
******************************
<bos> a man and a woman in festive costumes dancing <eos>
<bos> a man and a woman with <unk> on her head dance <eos>
<bos> a man and a woman wearing <unk> costu

In [59]:
for i in range(20):
    for j in range(5):
        print(intseq_to_caption(idx2token, train_captions[train_fns_list[i]][j]))

<bos> a black dog is running after a white dog in the snow <eos>
<bos> black dog chasing brown dog through snow <eos>
<bos> two dogs chase each other across the snowy ground <eos>
<bos> two dogs play together in the snow <eos>
<bos> two dogs running through a low lying body of water <eos>
<bos> a little baby plays croquet <eos>
<bos> a little girl plays croquet next to a truck <eos>
<bos> the child is playing <unk> by the truck <eos>
<bos> the kid is in front of a car with a put and a ball <eos>
<bos> the little boy is playing with a croquet <unk> and ball beside the car <eos>
<bos> a brown dog in the snow has something hot pink in its mouth <eos>
<bos> a brown dog in the snow holding a pink hat <eos>
<bos> a brown dog is holding a pink shirt in the snow <eos>
<bos> a dog is carrying something pink in its mouth while walking through the snow <eos>
<bos> a dog with something pink in its mouth is looking forward <eos>
<bos> a brown dog is running along a beach <eos>
<bos> a brown dog wea

In [106]:
from keras.models import load_model
model.save("test.h5")

del model  # deletes the existing model
model = load_model('test.h5')

/home/ubuntu/anaconda3/envs/tensorflow-3.5/lib/python3.5/site-packages/keras/models.py:255: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [109]:
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
import numpy as np

VGG16_model = VGG16(weights='imagenet', include_top=False, pooling='avg')

img_path = 'data/Arnav_Hankyu_Pulkit2.jpg'
img = image.load_img(img_path, target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

features = VGG16_model.predict(x)

In [110]:
print(generate_seq(features))

two men and a woman are posing for a picture <eos>
